<h1 style='color:#0000ec'>8 &nbsp;Estudo comparativo - Soluções numéricas e analítica</h1>

&nbsp; &nbsp; &nbsp; Importando bibliotecas <i><b>Numpy</b></i> e <i><b>Matplotlib</b></i>:

In [1]:
import numpy as np
import matplotlib.pyplot as plt

&nbsp; &nbsp; &nbsp; Importando bibliotecas <i><b>SciPy</b></i> :

In [2]:
import scipy.integrate as sciInt

&nbsp; &nbsp; &nbsp; Importando os modelos de <b>Malthus</b> e <b>Verhulst</b>:

In [3]:
import sys
sys.path.insert(0,'../python')

from modelo_malthus import modelo_malthus
from modelo_verhulst import modelo_verhulst

&nbsp; &nbsp; &nbsp; Importando os métodos de solução:

In [4]:
from sol_euler import sol_euler
from sol_euler_mod import sol_euler_mod
from sol_rk4 import sol_rk4

&nbsp; &nbsp; &nbsp; Declarando vetor contendo a população de Rio Grande, segundo o DATASUS, entre os anos de 2010 e 2012.

In [5]:
dados_datasus = [197228, 198049, 198842]

<h2 style='color:#0000b7' >&nbsp; 8.1 &nbsp;Obtendo aproximações</h2>

&nbsp; &nbsp; &nbsp; Declarando o passo de derivação <b>h</b>, a condição inicial <b>T0</b>, o tempo inicial <b>ti</b>, o tempo final <b>tf</b> e o intervalo de tempo <b>vt</b>:

In [6]:
h = 0.001 #Passo de derivação

P0 = 197228 #Condição inicial

ti = 0 #tempo inicial

tf = 12 #tempo final

vt = np.arange(ti,tf,h) #Intervalo de 10 à 21 anos com passo h

&nbsp; &nbsp; &nbsp; Obtendo solução analítica e segundo os métodos <b>LSODA</b>, <b>RK45</b>, <b>Euler</b>, <b>Euler modificado</b> e <b>RK4</b> para o modelo de <b>Malthus</b>:

In [7]:
malthus_analitico = 186488*np.exp(0.00407*vt)

malthus_sympy = 186488*np.exp(0.00407*vt)

malthus_lsoda = sciInt.odeint(modelo_malthus, y0=P0, t=vt, tfirst=True)

malthus_rk45 = sciInt.solve_ivp(modelo_malthus, t_span=(0,max(vt)), y0=[P0], t_eval=vt)

malthus_euler = sol_euler(vt,P0,'malthus')

malthus_euler_mod = sol_euler_mod(vt,P0,'malthus')

malthus_rk4 = sol_rk4(vt,P0,'malthus')

&nbsp; &nbsp; &nbsp; Obtendo solução analítica e segundo os métodos <b>LSODA</b>, <b>RK45</b>, <b>Euler</b>, <b>Euler modificado</b> e <b>RK4</b> para o modelo de <b>Verhulst</b>:

In [8]:
verhulst_analitico = (218858*197228*np.exp(0.04285*vt))/(218858+197228*(np.exp(0.04285*vt)-1))

verhulst_sympy = 218858/(1+0.109670026568236*np.exp(-0.04285*vt))

verhulst_lsoda = sciInt.odeint(modelo_verhulst, y0=P0, t=vt, tfirst=True)

verhulst_rk45 = sciInt.solve_ivp(modelo_verhulst, t_span=(0,max(vt)), y0=[P0], t_eval=vt)

verhulst_euler = sol_euler(vt,P0,'verhulst')

verhulst_euler_mod = sol_euler_mod(vt,P0,'verhulst')

verhulst_rk4 = sol_rk4(vt,P0,'verhulst')

In [9]:
print(malthus_rk45.y[0][0])

197228.0


<h2 style='color:#0000b7' >&nbsp; 8.2 &nbsp;Calculando o erro para 2010-2012</h2>

&nbsp; &nbsp; &nbsp; A seguir, escrevemos a função <i><b>erro_datasus(	&lt;dados do DATASUS>,	&lt;Vetor solução>)</b></i> o erro das soluções obtidas com relação a população fornecida pelo DATASUS.

In [10]:
%%writefile ../python/erro_datasus.py

import numpy as np

def erro_datasus(dados_datasus,solucoes,metodo):
    
    dados = [[0, 0, 0]]
    
    for i in range(0,3):
        
        if metodo=='lsoda':
            
            solucao = solucoes[i][0]
            
        elif metodo=='rk45':
            
            solucao = solucoes.y[0][i]
        
        else:
            
            solucao = solucoes[i]
            
        erro_hab = dados_datasus[i] - solucao
        #print(erro_hab)
        erro_perc = (erro_hab/dados_datasus[i])*100
        #Adicionando linhas à matriz (tabela)
        dados = np.vstack([dados,[solucao,round(erro_hab),str(round(erro_perc,2))+'%']])
        
    #remove a primeira linha da matriz
    dados = np.delete(dados, 0, 0)
    
    return dados

Overwriting ../python/erro_datasus.py


&nbsp; &nbsp; &nbsp; Importando função escrita em <b>[9]</b>

In [11]:
import sys
sys.path.insert(0, '../python')
from erro_datasus import erro_datasus

&nbsp; &nbsp; &nbsp; Atribuindo as aproximações à funções <i><b>erro_ibge</b></i>.

In [12]:
print(malthus_lsoda[0][0])

197228.0


In [13]:
err_malthus_analitico = erro_datasus(dados_datasus,malthus_analitico,'')

err_malthus_sympy = erro_datasus(dados_datasus,malthus_sympy,'')

err_malthus_lsoda = erro_datasus(dados_datasus,malthus_lsoda,'lsoda')

err_malthus_rk45 = erro_datasus(dados_datasus,malthus_rk45)

err_malthus_euler = erro_datasus(dados_datasus,malthus_euler)

err_malthus_euler_mod = erro_datasus(dados_datasus,malthus_euler_mod)

err_malthus_rk4 = erro_datasus(dados_datasus,malthus_euler_rk4)

TypeError: erro_datasus() missing 1 required positional argument: 'metodo'

&nbsp; &nbsp; &nbsp; Finalmente, exibimos os dados armazenados:

In [ ]:
fig, ax =plt.subplots(1,1)
column_labels=['Método', 'Erro absoluto', 'Erro percentual']
ax.axis('tight')
ax.axis('off')

print('Malthus:')
ax.table(cellText=dados_malthus,colLabels=column_labels,loc='center')
plt.show()

In [ ]:
fig, ax =plt.subplots(1,1)
column_labels=['Método', 'Erro absoluto', 'Erro percentual']
ax.axis('tight')
ax.axis('off')

print('Verhulst')
ax.table(cellText=dados_verhulst,colLabels=column_labels,loc='center')
plt.show()

In [ ]:
for i in range(0,3):
    print(i)